# Performing the Paired Difference T-test to test the blood pressure results before and after the intervention are statistically significant

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import scale

from scipy import stats
import researchpy as rp

In [3]:
bp_reading = pd.read_csv('datasets/blood_pressure.csv')
bp_reading.head()

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


In [4]:
bp_reading.shape

(120, 5)

In [6]:
bp_reading.describe().T

,count,mean,std,min,25%,50%,75%,max
patient,120.0,60.500000,34.785054,1.0,30.75,60.5,90.25,120.0
bp_before,120.0,156.450000,11.389845,138.0,147.00,154.5,164.00,185.0
bp_after,120.0,151.358333,14.177622,125.0,140.75,149.5,161.00,185.0


**Objective**

Using p-test to see if whether the difference in the mean differ after or before the intervention is statistically significant.

In [ ]:
bp_reading[['bp_before', 'bp_after']].boxplot9